In [1]:
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
df=pd.read_csv('creditcard.csv')
print(df.shape)
print(df.columns)


(284807, 31)
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [3]:
print("Time:\n")
print(df['Time'].describe())

Time:

count    284807.000000
mean      94813.859575
std       47488.145955
min           0.000000
25%       54201.500000
50%       84692.000000
75%      139320.500000
max      172792.000000
Name: Time, dtype: float64


In [4]:
print("Amount:\n")
print(df['Amount'].describe())

Amount:

count    284807.000000
mean         88.349619
std         250.120109
min           0.000000
25%           5.600000
50%          22.000000
75%          77.165000
max       25691.160000
Name: Amount, dtype: float64


In [5]:
print("Class:\n")
print(df['Class'].describe())
print(df['Class'].value_counts())

Class:

count    284807.000000
mean          0.001727
std           0.041527
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: Class, dtype: float64
Class
0    284315
1       492
Name: count, dtype: int64


In [6]:
print(df)

            Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0       0.462388  0.239599  0.098698  0.363787  ... -0.01830

In [7]:
## graph construction
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx


In [8]:
fraud_data=df[df['Class']==1]
normal_data=df[df['Class']==0].sample(n=len(fraud_data)*10,random_state=42)
print(fraud_data.shape)
print(normal_data.shape)
selected_df=pd.concat([normal_data,fraud_data])
from sklearn.preprocessing import StandardScaler

# 假設 df 是你的 DataFrame，columns 是要標準化的欄位名稱列表
columns_to_normalize = ['Time', 'Amount']  # 填入要標準化的欄位名稱

# 初始化 StandardScaler 物件
scaler = StandardScaler()

# 對指定列進行標準化
selected_df[columns_to_normalize] = scaler.fit_transform(selected_df[columns_to_normalize])
features=selected_df.drop("Class",axis=1)
cos_sim=cosine_similarity(features)
np.save("cos_sim.npy",cos_sim)
print("Cosine similarity calculation completed and saved.")


(492, 31)
(4920, 31)
Cosine similarity calculation completed and saved.


In [9]:
cos_sim=np.load("cos_sim.npy")
print(cos_sim)
print(cos_sim.shape)

[[ 1.         -0.10474847 -0.17063587 ...  0.00880987  0.19544622
   0.52532881]
 [-0.10474847  1.          0.35448425 ... -0.07256405 -0.13313727
  -0.33534979]
 [-0.17063587  0.35448425  1.         ... -0.11640662 -0.14318716
  -0.26465308]
 ...
 [ 0.00880987 -0.07256405 -0.11640662 ...  1.          0.89628544
   0.18719061]
 [ 0.19544622 -0.13313727 -0.14318716 ...  0.89628544  1.
   0.22006893]
 [ 0.52532881 -0.33534979 -0.26465308 ...  0.18719061  0.22006893
   1.        ]]
(5412, 5412)


In [10]:
# 計算每一列的統計量
max_values = np.max(cos_sim, axis=1)
min_values = np.min(cos_sim, axis=1)
median_values = np.median(cos_sim, axis=1)
mean_values = np.mean(cos_sim, axis=1)
std_values = np.std(cos_sim, axis=1)

# 對這些統計量再次計算統計量
stats_max = np.max(max_values)
stats_min = np.min(min_values)
stats_median = np.median(median_values)
stats_mean = np.mean(mean_values)
stats_std = np.std(std_values)

# 印出結果
print("Statistics of row statistics:")
print("  Max:", stats_max)
print("  Min:", stats_min)
print("  Median:", stats_median)
print("  Mean:", stats_mean)
print("  Std:", stats_std)

Statistics of row statistics:
  Max: 1.000000000000001
  Min: -0.8610622522995957
  Median: -0.020403275449845982
  Mean: 0.007676525874176422
  Std: 0.031593458544467296


In [11]:
G=nx.Graph()
nodes_list = []
for i in range(cos_sim.shape[0]):
    nodes_list.append((i, {'feature': features.values[i]}))
G.add_nodes_from(nodes_list)


for i in tqdm(range(cos_sim.shape[0]),desc="Graph Construction..."):
    for j in range(i+1,cos_sim.shape[1]):
        if cos_sim[i,j]>stats_median+stats_std*8:
            G.add_edge(i,j)
print("Number of nodes:",G.number_of_nodes())
print("Number of edges:",G.number_of_edges())
density=2*G.number_of_edges()/(G.number_of_nodes()*(G.number_of_nodes()-1))
print(density)

Graph Construction...: 100%|██████████| 5412/5412 [00:07<00:00, 772.32it/s] 

Number of nodes: 5412
Number of edges: 2495539
0.1704350981951714


In [12]:
from collections import Counter
selected_df.reset_index(drop=True,inplace=True)
# print(selected_df['Class'])

for node in G.nodes():
    # print(node)
    node_class=selected_df.loc[node,'Class']
    G.nodes[node]['Class']=node_class

# 檢查每個節點的 Class 屬性

# 使用 nx.degree() 函數計算每個節點的度
node_degrees = dict(nx.degree(G))

# 使用 Counter 類對節點的度進行統計
# degree_counts = Counter(node_degrees.values())

# # 印出結果
# print("Degree statistics:")
# for degree, count in sorted(degree_counts.items()):
#     print("Degree:", degree, "Count:", count)

# # 找出所有類別為 1 的節點
# class_1_nodes = [node for node, data in G.nodes(data=True) if data.get('Class') == 1]

# # 計算這些節點的度
# class_1_degrees = {node: G.degree(node) for node in class_1_nodes}

# # 印出結果

# for node, degree in class_1_degrees.items():
#     print("Node:", node, "Degree:", degree)
# print(len(class_1_degrees.items()))

In [13]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
import networkx as nx
import numpy as np
# 轉換為 PyTorch 張量
edge_index = torch.tensor(list(G.edges)).t().contiguous()
x = torch.tensor([node[1] for node in G.nodes.data('feature')], dtype=torch.float32)
# 精度被吃掉？？
print(x[0])
y = torch.tensor([node[1] for node in G.nodes.data('Class')], dtype=torch.long)  # 節點的標籤

data = Data(x=x, edge_index=edge_index, y=y)

/home/p76121194/miniconda3/envs/dl_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([-0.2365,  1.3145,  0.5906, -0.6666,  0.7166,  0.3020, -1.1255,  0.3889,
        -0.2884, -0.1321, -0.5977, -0.3253, -0.2164,  0.0842, -1.0546,  0.9679,
         0.6012,  0.6311,  0.2951, -0.1362, -0.0580, -0.1703, -0.4297, -0.1413,
        -0.2002,  0.6395,  0.3995, -0.0343,  0.0317, -0.3871])


/tmp/ipykernel_61633/4107390831.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/torch/csrc/utils/tensor_new.cpp:230.)
  x = torch.tensor([node[1] for node in G.nodes.data('feature')], dtype=torch.float32)


In [14]:
from torch_geometric.nn import GATConv
import torch.nn.functional as F
class Encoder_GAT(torch.nn.Module):
    def __init__(self, num_heads, input_dim, hidden_dim, output_dim, num_layers):
        super(Encoder_GAT, self).__init__()
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers-1):
            if i:
                conv = GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
            else:
                conv = GATConv(input_dim, hidden_dim, heads=num_heads)

            self.convs.append(conv)
        conv = GATConv(hidden_dim * num_heads, output_dim)
        self.convs.append(conv)

    def forward(self, x, edge_index):
        xs = []
        for i in range(self.num_layers):
            x = F.relu(self.convs[i](x, edge_index))
        return x

In [15]:
import torch.nn as nn
import torch.nn.functional as F
class myGNN(nn.Module):
    def __init__(self, enc_num_heads, enc_input_dim, enc_hidden_dim, enc_num_layers, linear_output_dim):
        super(myGNN, self).__init__()
        # GAT(input、output大小一樣)
        self.encoder_neighbor = Encoder_GAT(enc_num_heads, enc_input_dim, enc_hidden_dim, enc_input_dim, enc_num_layers)
        # linear層
        self.proj_head_neighbor = nn.Linear(enc_input_dim, linear_output_dim)
        self.proj_head_ego = nn.Linear(enc_input_dim, linear_output_dim)

        self.init_emb()
    # embedding初始化
    def init_emb(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight.data)
                # BIASE初始化為0
                if m.bias is not None:
                    m.bias.data.fill_(0.0)
    @staticmethod
    def negative_sample(h_ego, h_neighbor):
        # 返回一个0~n-1的数组，随机打散的
        perm = torch.randperm(h_ego.shape[0])
        # 打亂，作ego-ego negative
        h_ego_neg = h_ego[perm]
        # 打亂，作ego-neigbor negative
        h_neighbor_neg = h_neighbor[perm]
        return h_ego_neg, h_neighbor_neg

    # @staticmethod
    # def cosine_similarity(x1, x2):
    #     return torch.div(torch.sum(x1 * x2,0),torch.sqrt(torch.sum(torch.pow(x1,2),0))* torch.sqrt(torch.sum(torch.pow(x2,2),0))).item()

    @staticmethod
    def discriminator(x1, x2):
        return -1 * F.cosine_similarity(x1, x2, dim=1).unsqueeze(0)

    def forward(self, x, edge_index):
        # GAT
        h_neighbor = self.encoder_neighbor(x, edge_index)
        h_neighbor = self.proj_head_neighbor(h_neighbor)
        # linear
        h_ego = self.proj_head_ego(x)

        return h_ego, h_neighbor

In [16]:
import time
from sklearn.metrics import roc_auc_score
import json
import copy
import os
import numpy as np
import torch

def rescale(x):
    return (x + 1) / 2

def train_model(args, data, model, optimizer, loss_function):
    stats = {
        "best_loss": 1e9,
        "best_epoch": -1,
    }
    model.train()

    label_ones =  torch.ones(1, data.x.shape[0]).to(args["device"])
    label_zeros = torch.zeros(1, data.x.shape[0]).to(args["device"])

    for epoch in tqdm(range(args['num_epoch'])):
        optimizer.zero_grad()
        loss_pos = 0
        data = data.to(args['device'])
        # forward(gat+linear)
        h_ego, h_neighbor = model(data.x, data.edge_index)
        h_ego_neg, h_neighbor_neg  = model.negative_sample(h_ego, h_neighbor)
        # 算 -c
        c_neighbor_pos = model.discriminator(h_ego, h_neighbor)
        c_neighbor_neg = model.discriminator(h_ego, h_neighbor_neg)
        c_ego_neg = model.discriminator(h_ego, h_ego_neg)
        # rescal(x) = (x-(-1)) / 2，使介於0~1(原介於-1~1)
        score_pos = rescale(c_neighbor_pos)
        score_aug = rescale(c_neighbor_neg)
        score_nod = rescale(c_ego_neg)
        # BCE loss
        # ego-neighbor postive, ego-neighbor negative, ego-ego negative
        loss_pos = loss_function(score_pos, label_zeros)
        loss_aug = loss_function(score_aug, label_ones)
        loss_nod = loss_function(score_nod, label_ones)
        loss_sum = loss_pos \
              + args['alpha'] * loss_aug \
              + args['gamma'] * loss_nod

        loss_sum.backward()
        # 只用postive判斷好壞
        if loss_pos < stats["best_loss"]:
            stats["best_loss"] = loss_pos.item()
            stats["best_epoch"] = epoch
            torch.save(model.state_dict(), args['state_path'])
        optimizer.step()


    return stats

def eval_model(args, data, model):
    model.eval()
    with torch.no_grad():
        data = data.to(args["device"])
        h_ego, h_neighbor = model(data.x, data.edge_index)
        c_neighbor_pos = model.discriminator(h_ego, h_neighbor)
        
        y_true = (data.y).detach().cpu().tolist()
        y_score = c_neighbor_pos.squeeze().detach().cpu().tolist()
        auc = roc_auc_score(y_true, y_score)
    return auc


In [17]:
import random

def set_random_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def run_experiment(args, data):
    set_random_seeds(args['seed'])
    # Create model
    model = myGNN(args['enc_num_heads'], args['enc_input_dim'], args['enc_hidden_dim'],  args['enc_num_layers'], args["linear_output_dim"])
    model.to(args['device'])
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=args['lr'],
                                 weight_decay=args['weight_decay'])
    loss_function = torch.nn.BCELoss()
    # train
    stats = train_model(
        args, data, model, optimizer, loss_function
    )
    # eval
    model.load_state_dict(torch.load(args["state_path"]))
    auc = eval_model(args, data, model)
    stats["AUC"] = auc

    return model, stats

In [18]:
args = {"lr": 5e-4, 
        "alpha": 0.3, 
        "gamma": 0.4, 
        "state_path": "model.pkl", 
        "device": "cuda:0", 
        "seed": 1, 
        "num_epoch": 3, 
        "weight_decay": 0.0, 
        "enc_num_heads": 2, 
        "enc_input_dim":data.x.shape[1], 
        "enc_hidden_dim": 32, 
        "linear_output_dim": 64,
        "enc_num_layers":2,
       }

In [19]:

model, stats = run_experiment(args, data)
print(stats)

100%|██████████| 3/3 [00:00<00:00,  5.59it/s]

{'best_loss': 0.6684134602546692, 'best_epoch': 2, 'AUC': 0.6028277645581334}
